In [1]:
%pip install pyannote.audio
%pip install numpy==1.26

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   -------

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

HUGGING_FACE_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

In [11]:
#instantiate the pipeline
from pyannote.audio import Pipeline

pipeline = Pipeline.from_pretrained(
  "pyannote/speaker-diarization-3.1",
  use_auth_token=HUGGING_FACE_TOKEN
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\workspace\agent_work1\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\torch\pyannote\models--pyannote--segmentation-3.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: h

In [12]:
import torch

#cuda가 사용 가능한 경우 cuda를 사용하도록 설정
if torch.cuda.is_available():
    pipeline.to(torch.device("cuda"))
    print('cuda is available')
else:
    print('cuda is not available')

cuda is not available


In [15]:
#run the pipeline on an audio file
#diarization = pipeline("audio.wav")
diarization = pipeline("./audio/싼기타_비싼기타.mp3")

#dump the diarization output to disk using RTTM format
with open("싼기타_비싼기타.rttm", "w", encoding='utf-8') as rttm:
    diarization.write_rttm(rttm)

c:\workspace\agent_work1\venv\Lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)
c:\workspace\agent_work1\venv\Lib\site-packages\pyannote\audio\core\io.py:85: UserWarning: torchaudio._backend.utils.info has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  info = torchaudio.info(file["audio"], backend=backend)
c:\workspace\agent_work1\venv\Lib\site-packages\torchaudio\

In [18]:
#RTTM을 CSV로 변환
import pandas as pd
rttm_path = "./싼기타_비싼기타.rttm"

df_rttm = pd.read_csv(
    rttm_path,      # rttm 파일 경로
    sep=' ',        # 구분자는 띄어쓰기
    header=None,    # 헤더는 없음
    names=['type', 'file', 'chnl', 'start', 'duration', 'C1', 'C2', 'speaker_id', 'C3', 'C4'] 
)

display(df_rttm)

,type,file,chnl,start,duration,C1,C2,speaker_id,C3,C4
0,SPEAKER,싼기타_비싼기타,1,0.993,5.805,NaN,NaN,SPEAKER_01,NaN,NaN
1,SPEAKER,싼기타_비싼기타,1,7.405,3.983,NaN,NaN,SPEAKER_01,NaN,NaN
2,SPEAKER,싼기타_비싼기타,1,11.759,4.927,NaN,NaN,SPEAKER_01,NaN,NaN
3,SPEAKER,싼기타_비싼기타,1,17.210,10.665,NaN,NaN,SPEAKER_01,NaN,NaN
4,SPEAKER,싼기타_비싼기타,1,28.668,1.536,NaN,NaN,SPEAKER_01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
83,SPEAKER,싼기타_비싼기타,1,414.481,2.970,NaN,NaN,SPEAKER_00,NaN,NaN
84,SPEAKER,싼기타_비싼기타,1,417.755,3.476,NaN,NaN,SPEAKER_01,NaN,NaN
85,SPEAKER,싼기타_비싼기타,1,423.644,0.776,NaN,NaN,SPEAKER_00,NaN,NaN
86,SPEAKER,싼기타_비싼기타,1,424.741,3.527,NaN,NaN,SPEAKER_00,NaN,NaN


In [19]:
#start + duration을 end로 변환
df_rttm['end'] = df_rttm['start'] + df_rttm['duration']

display(df_rttm)

,type,file,chnl,start,duration,C1,C2,speaker_id,C3,C4,end
0,SPEAKER,싼기타_비싼기타,1,0.993,5.805,NaN,NaN,SPEAKER_01,NaN,NaN,6.798
1,SPEAKER,싼기타_비싼기타,1,7.405,3.983,NaN,NaN,SPEAKER_01,NaN,NaN,11.388
2,SPEAKER,싼기타_비싼기타,1,11.759,4.927,NaN,NaN,SPEAKER_01,NaN,NaN,16.686
3,SPEAKER,싼기타_비싼기타,1,17.210,10.665,NaN,NaN,SPEAKER_01,NaN,NaN,27.875
4,SPEAKER,싼기타_비싼기타,1,28.668,1.536,NaN,NaN,SPEAKER_01,NaN,NaN,30.204
...,...,...,...,...,...,...,...,...,...,...,...
83,SPEAKER,싼기타_비싼기타,1,414.481,2.970,NaN,NaN,SPEAKER_00,NaN,NaN,417.451
84,SPEAKER,싼기타_비싼기타,1,417.755,3.476,NaN,NaN,SPEAKER_01,NaN,NaN,421.231
85,SPEAKER,싼기타_비싼기타,1,423.644,0.776,NaN,NaN,SPEAKER_00,NaN,NaN,424.420
86,SPEAKER,싼기타_비싼기타,1,424.741,3.527,NaN,NaN,SPEAKER_00,NaN,NaN,428.268


In [20]:
df_rttm["number"] = None    # number 열 만들고 None으로 초기화
df_rttm.at[0, "number"] = 0

display(df_rttm)
for i in range(1, len(df_rttm)):
    if df_rttm.at[i, "speaker_id"] != df_rttm.at[i-1, "speaker_id"]:
        df_rttm.at[i, "number"] = df_rttm.at[i-1, "number"] + 1
    else:
        df_rttm.at[i, "number"] = df_rttm.at[i-1, "number"]

display(df_rttm.head(10))
df_rttm_grouped = df_rttm.groupby("number").agg(
    start=pd.NamedAgg(column='start', aggfunc='min'),
    end=pd.NamedAgg(column='end', aggfunc='max'),
    speaker_id=pd.NamedAgg(column='speaker_id', aggfunc='first')
)

display(df_rttm_grouped)

,type,file,chnl,start,duration,C1,C2,speaker_id,C3,C4,end,number
0,SPEAKER,싼기타_비싼기타,1,0.993,5.805,NaN,NaN,SPEAKER_01,NaN,NaN,6.798,0
1,SPEAKER,싼기타_비싼기타,1,7.405,3.983,NaN,NaN,SPEAKER_01,NaN,NaN,11.388,None
2,SPEAKER,싼기타_비싼기타,1,11.759,4.927,NaN,NaN,SPEAKER_01,NaN,NaN,16.686,None
3,SPEAKER,싼기타_비싼기타,1,17.210,10.665,NaN,NaN,SPEAKER_01,NaN,NaN,27.875,None
4,SPEAKER,싼기타_비싼기타,1,28.668,1.536,NaN,NaN,SPEAKER_01,NaN,NaN,30.204,None
...,...,...,...,...,...,...,...,...,...,...,...,...
83,SPEAKER,싼기타_비싼기타,1,414.481,2.970,NaN,NaN,SPEAKER_00,NaN,NaN,417.451,None
84,SPEAKER,싼기타_비싼기타,1,417.755,3.476,NaN,NaN,SPEAKER_01,NaN,NaN,421.231,None
85,SPEAKER,싼기타_비싼기타,1,423.644,0.776,NaN,NaN,SPEAKER_00,NaN,NaN,424.420,None
86,SPEAKER,싼기타_비싼기타,1,424.741,3.527,NaN,NaN,SPEAKER_00,NaN,NaN,428.268,None


,type,file,chnl,start,duration,C1,C2,speaker_id,C3,C4,end,number
0,SPEAKER,싼기타_비싼기타,1,0.993,5.805,NaN,NaN,SPEAKER_01,NaN,NaN,6.798,0
1,SPEAKER,싼기타_비싼기타,1,7.405,3.983,NaN,NaN,SPEAKER_01,NaN,NaN,11.388,0
2,SPEAKER,싼기타_비싼기타,1,11.759,4.927,NaN,NaN,SPEAKER_01,NaN,NaN,16.686,0
3,SPEAKER,싼기타_비싼기타,1,17.210,10.665,NaN,NaN,SPEAKER_01,NaN,NaN,27.875,0
4,SPEAKER,싼기타_비싼기타,1,28.668,1.536,NaN,NaN,SPEAKER_01,NaN,NaN,30.204,0
5,SPEAKER,싼기타_비싼기타,1,32.414,0.759,NaN,NaN,SPEAKER_00,NaN,NaN,33.173,1
6,SPEAKER,싼기타_비싼기타,1,33.545,3.561,NaN,NaN,SPEAKER_00,NaN,NaN,37.106,1
7,SPEAKER,싼기타_비싼기타,1,37.628,3.763,NaN,NaN,SPEAKER_00,NaN,NaN,41.391,1
8,SPEAKER,싼기타_비싼기타,1,41.611,1.097,NaN,NaN,SPEAKER_00,NaN,NaN,42.708,1
9,SPEAKER,싼기타_비싼기타,1,41.645,0.810,NaN,NaN,SPEAKER_01,NaN,NaN,42.455,2


,start,end,speaker_id
number,,,
0,0.993,30.204,SPEAKER_01
1,32.414,42.708,SPEAKER_00
2,41.645,44.024,SPEAKER_01
3,45.813,67.109,SPEAKER_00
4,67.227,82.786,SPEAKER_01
5,84.659,102.564,SPEAKER_00
6,103.492,117.532,SPEAKER_01
7,119.759,138.676,SPEAKER_00
8,139.351,168.967,SPEAKER_01


In [21]:
df_rttm_grouped["duration"] = df_rttm_grouped["end"] - df_rttm_grouped["start"]
df_rttm_grouped = df_rttm_grouped.reset_index(drop=True)
display(df_rttm_grouped)

,start,end,speaker_id,duration
0,0.993,30.204,SPEAKER_01,29.211
1,32.414,42.708,SPEAKER_00,10.294
2,41.645,44.024,SPEAKER_01,2.379
3,45.813,67.109,SPEAKER_00,21.296
4,67.227,82.786,SPEAKER_01,15.559
5,84.659,102.564,SPEAKER_00,17.905
6,103.492,117.532,SPEAKER_01,14.040
7,119.759,138.676,SPEAKER_00,18.917
8,139.351,168.967,SPEAKER_01,29.616
9,170.907,192.321,SPEAKER_00,21.414


In [24]:
df_rttm_grouped.to_csv(
    "싼기타_비싼기타_rttm.csv",
    sep=',',
    index=False
)